In [ ]:
import os
import re
import glob
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

import os, sys
from interference_model.quantification import get_interference_rms

# For bigfiles
from io import TextIOWrapper
from zipfile import ZipFile

from pathlib import Path

import colorama
from colorama import Fore, Style
import seaborn as sns

In [ ]:
# Color rules (based on https://personal.sron.nl/~pault/)
GREEN = "#117733"
TEAL  = "#44AA99"
CYAN = "#88CCEE"
OLIVE = "#CCBB43"
SAND = "#DDCC77"
ROSE   = "#EE6677"
BLUE = "#88CCEE"
MAGENTA = "#AA3377"
GREY = GRAY = "#DDDDDD"

def set_font(size):
    text_font_size = size
    marker_font_size = size
    label_font_size = size
    axes_font_size = size

    plt.rc('text', usetex=True)
    plt.rc('pdf', use14corefonts=True, fonttype=42)
    plt.rc('ps', useafm=True)
    plt.rc('font', size=text_font_size, weight="bold", family='serif', serif='cm10')
    plt.rc('axes', labelsize=axes_font_size,labelweight="bold")    
    plt.rc('xtick', labelsize=label_font_size)    
    plt.rc('ytick', labelsize=label_font_size)    
    plt.rc('legend', fontsize=label_font_size)  
    plt.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
    plt.rcParams['text.latex.preamble'] = r'\boldmath'
set_font(21)

def save_plot(name):
    plt.savefig(f'./plots/{name}')

def bold(text):
    return r'\textbf{' + text + r'}'
    
def stdef(vals):
    avg = sum(vals) / len(vals)
    sumderiv = 0;
    for val in vals:
        sumderiv = sumderiv + val * val
    sumderivavg = sumderiv / (len(vals) - 1)
    return math.sqrt(sumderivavg - (avg*avg))


def avg(vals):
    return sum(vals)/len(vals)

In [ ]:
lbaf0_pattern = r'\\'
lbaf2_pattern = '.'
set_font(21)

In [ ]:
def parse_fio_data(data_path, data):
    if not os.path.exists(f'{data_path}') or \
            os.listdir(f'{data_path}') == []:
        print(f"No data in {data_path}")
        return 0

    for file in glob.glob(f'{data_path}/*'):
        if "bw" in file:
            continue
        with open(file, 'r') as f:
            for index, line in enumerate(f, 1):
                # Removing all fio logs in json file by finding first {
                if line.split()[0] == "{":
                    rows = f.readlines()
                    with open(os.path.join(os.getcwd(), "temp.json"), 'w+') as temp:
                        temp.write(line)
                        temp.writelines(rows)
                    break

        with open(os.path.join(os.getcwd(), "temp.json"), 'r') as temp:
            data[file] = dict()
            data[file] = json.load(temp)
            os.remove(os.path.join(os.getcwd(), "temp.json"))

    return 1

In [ ]:
data = dict()
parse_fio_data(f"../reset-on-append-interference/data", data)

queue_depths = np.arange(1, 8)

write100 = [None] * len(queue_depths)
write100_iops = [None] * len(queue_depths)
write99 = [None] * len(queue_depths)
write99_iops = [None] * len(queue_depths)
write95 = [None] * len(queue_depths)
write95_iops = [None] * len(queue_depths)
write90 = [None] * len(queue_depths)
write90_iops = [None] * len(queue_depths)
write75 = [None] * len(queue_depths)
write75_iops = [None] * len(queue_depths)
write50 = [None] * len(queue_depths)
write50_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    numjobs = int(re.search(r'\d+', key).group())
    if numjobs > 7:
        continue # We skip 14 as it turns out we were bottlenecked by not enough cpu cores, giving inaccurate results
    else:
        x = numjobs - 1
    if 'aflow_100' in key:
        write100[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write100_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'aflow_99' in key:
        write99[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write99_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_95' in key:
        write95[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write95_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_90' in key:
        write90[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write90_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_75' in key:
        write75[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write75_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_50' in key:
        write50[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write50_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000

print(write100_iops)
print(f"Interference RMS 0% - 50% {get_interference_rms(write100_iops, write50_iops, write100, write50)}")

fig, ax = plt.subplots()

ax.plot(write100_iops, write100, markersize = 4, marker = '>', label="  0% reset", color="#000000", linewidth=2)
ax.plot(write99_iops, write99, markersize = 4, marker = 'x', label="  1% reset"  , color=CYAN, linewidth=2)
ax.plot(write95_iops, write95, markersize = 4, marker = 'o', label="  5% reset"  , color=MAGENTA, linewidth=2)
ax.plot(write90_iops, write90, markersize = 4, marker = '<', label=" 10% reset"  , color=OLIVE, linewidth=2)
ax.plot(write75_iops, write75, markersize = 4, marker = '^',  label=" 25% reset" , color=ROSE, linewidth=2)
ax.plot(write50_iops, write50, markersize = 4, marker = '*', label=" 50% reset"  , color=GREEN, linewidth=2)

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
# ax.legend(loc='best', handles=handles)
# ax.legend(loc='best')
ax.set_ylim(bottom=0, top=150)
ax.set_xlim(left=0, right=175)
ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')

ax.xaxis.set_ticks(np.arange(0, 200, 25))
ax.yaxis.set_ticks(np.arange(0, 175, 25))

plt.savefig('./plots/interference-micro-reset-on-append.pdf', bbox_inches='tight')

In [ ]:
# Reset on read
data = dict()
parse_fio_data(f"../reset-on-read-interference/data", data)

queue_depths = np.arange(1, 9)

read100 = [None] * len(queue_depths)
read100_iops = [None] * len(queue_depths)
read99 = [None] * len(queue_depths)
read99_iops = [None] * len(queue_depths)
read95 = [None] * len(queue_depths)
read95_iops = [None] * len(queue_depths)
read90 = [None] * len(queue_depths)
read90_iops = [None] * len(queue_depths)
read75 = [None] * len(queue_depths)
read75_iops = [None] * len(queue_depths)
read50 = [None] * len(queue_depths)
read50_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    x = int(math.log2(int(value["jobs"][1]["job options"]["iodepth"])))
    if 'rflow_100' in key:
        read100[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read100_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_99' in key:
        read99[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read99_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_95' in key:
        read95[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read95_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_90' in key:
        read90[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read90_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_75' in key:
        read75[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read75_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_50' in key:
        read50[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read50_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000

print(f"Interference RMS 0% - 50% {get_interference_rms(read100_iops,read50_iops, read100, read50)}")
# print(f"Interference EMD 0% - 50% {get_emd(read100_iops,read50_iops, read100, read50)}")

fig, ax = plt.subplots()

ax.plot(read100_iops, read100, markersize = 4, marker = '>', label="  0% reset",color="#000000", linewidth=2)
ax.plot(read99_iops, read99, markersize = 4, marker = 'x', label="  1% reset", color=CYAN, linewidth=2)
ax.plot(read95_iops, read95, markersize = 4, marker = 'o', label="  5% reset", color=MAGENTA, linewidth=2)
ax.plot(read90_iops, read90, markersize = 4, marker = '<', label=" 10% reset", color=OLIVE, linewidth=2)
ax.plot(read75_iops, read75, markersize = 4, marker = '^',  label=" 25% reset", color=ROSE, linewidth=2)
ax.plot(read50_iops, read50, markersize = 4, marker = '*', label=" 50% reset", color=GREEN, linewidth=2)

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
# ax.legend(loc='best', handles=handles)
# ax.legend(loc='best')
ax.set_ylim(bottom=0,top=1000)
ax.set_xlim(left=0,right=300)
ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')

ax.xaxis.set_ticks(np.arange(0, 320, 50))
ax.yaxis.set_ticks(np.arange(0, 1200, 200))

plt.savefig('./plots/interference-micro-reset-on-read.pdf', bbox_inches='tight')

In [ ]:
# Reset on write
data = dict()
parse_fio_data(f"../reset-on-write-interference/data", data)

queue_depths = np.arange(1, 8)

write100 = [None] * len(queue_depths)
write100_iops = [None] * len(queue_depths)
write99 = [None] * len(queue_depths)
write99_iops = [None] * len(queue_depths)
write95 = [None] * len(queue_depths)
write95_iops = [None] * len(queue_depths)
write90 = [None] * len(queue_depths)
write90_iops = [None] * len(queue_depths)
write75 = [None] * len(queue_depths)
write75_iops = [None] * len(queue_depths)
write50 = [None] * len(queue_depths)
write50_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    numjobs = int(re.search(r'\d+', key).group())
    if numjobs > 7:
        continue # We skip 14 as it turns out we were bottlenecked by not enough cpu cores, giving inaccurate results
    else:
        x = numjobs - 1
    if 'wflow_100' in key:
        write100[x] = value["jobs"][0]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write100_iops[x] = value["jobs"][0]["write"]["iops_mean"]/1000
    elif 'wflow_99' in key:
        write99[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write99_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000
    elif 'wflow_95' in key:
        write95[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write95_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000
    elif 'wflow_90' in key:
        write90[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write90_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000
    elif 'wflow_75' in key:
        write75[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write75_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000
    elif 'wflow_50' in key:
        write50[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write50_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000

print(f"Interference RMS 0% - 50% {get_interference_rms(write100_iops, write50_iops, write100, write50)}")

fig, ax = plt.subplots()

ax.plot(write100_iops, write100, markersize = 4, marker = '>', label=r'\textbf{0\%}', color="#000000", linewidth=2)
ax.plot(write99_iops, write99, markersize = 4, marker = 'x', label=r'\textbf{1\%}', color=CYAN, linewidth=2)
ax.plot(write95_iops, write95, markersize = 4, marker = 'o', label=r'\textbf{5\%}', color=MAGENTA, linewidth=2)
ax.plot(write90_iops, write90, markersize = 4, marker = '<', label=r'\textbf{10\%}', color=OLIVE, linewidth=2)
ax.plot(write75_iops, write75, markersize = 4, marker = '^',  label=r'\textbf{25\%}', color=ROSE, linewidth=2)
ax.plot(write50_iops, write50, markersize = 4, marker = '*', label=r'\textbf{50\%}', color=GREEN, linewidth=2)

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
#ax.legend(loc='best', handles=handles)
ax.legend(loc='best', title=r'\textbf{Reset \%}')
ax.set_ylim(bottom=0, top=140)
ax.set_xlim(left=0, right=150)
ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')

ax.xaxis.set_ticks(np.arange(0, 200, 25))
ax.yaxis.set_ticks(np.arange(0, 175, 25))

plt.savefig('./plots/interference-micro-reset-on-write.pdf', bbox_inches='tight')

In [ ]:
# Reset on write
data = dict()
parse_fio_data(f"../reset-on-write-interference/data", data)

queue_depths = np.arange(1, 8)

write100 = [None] * len(queue_depths)
write100_iops = [None] * len(queue_depths)
write99 = [None] * len(queue_depths)
write99_iops = [None] * len(queue_depths)
write95 = [None] * len(queue_depths)
write95_iops = [None] * len(queue_depths)
write90 = [None] * len(queue_depths)
write90_iops = [None] * len(queue_depths)
write75 = [None] * len(queue_depths)
write75_iops = [None] * len(queue_depths)
write50 = [None] * len(queue_depths)
write50_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    numjobs = int(re.search(r'\d+', key).group())
    if numjobs > 7:
        continue # We skip 14 as it turns out we were bottlenecked by not enough cpu cores, giving inaccurate results
    else:
        x = numjobs - 1
    if 'wflow_100' in key:
        write100[x] = value["jobs"][0]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write100_iops[x] = value["jobs"][0]["write"]["iops_mean"]/1000
    elif 'wflow_99' in key:
        write99[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write99_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000
    elif 'wflow_95' in key:
        write95[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write95_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000
    elif 'wflow_90' in key:
        write90[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write90_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000
    elif 'wflow_75' in key:
        write75[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write75_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000
    elif 'wflow_50' in key:
        write50[x] = value["jobs"][1]["write"]["lat_ns"]["percentile"]["95.000000"]/1000
        write50_iops[x] = value["jobs"][1]["write"]["iops_mean"]/1000

print(f"Interference RMS 0% - 50% {get_interference_rms(write100_iops, write50_iops, write100, write50)}")

fig, ax = plt.subplots()

ax.plot(write100_iops, write100, markersize = 4, marker = '>', label=r'\textbf{0\%}', color="#000000", linewidth=2)
ax.plot(write99_iops, write99, markersize = 4, marker = 'x', label=r'\textbf{1\%}', color=CYAN, linewidth=2)
ax.plot(write95_iops, write95, markersize = 4, marker = 'o', label=r'\textbf{5\%}', color=MAGENTA, linewidth=2)
ax.plot(write90_iops, write90, markersize = 4, marker = '<', label=r'\textbf{10\%}', color=OLIVE, linewidth=2)
ax.plot(write75_iops, write75, markersize = 4, marker = '^',  label=r'\textbf{25\%}', color=ROSE, linewidth=2)
ax.plot(write50_iops, write50, markersize = 4, marker = '*', label=r'\textbf{50\%}', color=GREEN, linewidth=2)

write50_iops_namespaces = [62.486982759, 82.841931034, 98.32489655200001, 100.31720689699999, 100.482517241, 101.9595, 101.767558912]
write50_namespaces = [16.512, 18.816, 21.12, 53.504, 81.408, 104.96, 114.176]
ax.plot(write50_iops_namespaces, write50_namespaces, markersize = 4, marker = '*', label=r'\textbf{50\% NS}', 
        color='#882255', linewidth=2, linestyle='--')

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
#ax.legend(loc='best', handles=handles)
ax.legend(loc=(0,0.21))
ax.set_ylim(bottom=0, top=140)
ax.set_xlim(left=0, right=150)
ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')

ax.xaxis.set_ticks(np.arange(0, 200, 25))
ax.yaxis.set_ticks(np.arange(0, 175, 25))
plt.savefig('./plots/interference-micro-reset-on-write.pdf', bbox_inches='tight')

In [ ]:
data = dict()

def parse_fio_data(data_path, data):
    if not os.path.exists(f'{data_path}') or \
            os.listdir(f'{data_path}') == []:
        print(f"No data in {data_path}")
        return 0

    for file in glob.glob(f'{data_path}/*'):
        if "bw" in file:
            continue
        with open(file, 'r') as f:
            for index, line in enumerate(f, 1):
                # Removing all fio logs in json file by finding first {
                if line.split() == []:
                    pass
                elif line.split()[0] == "{":
                    rows = f.readlines()
                    with open(os.path.join(os.getcwd(), "temp.json"), 'w+') as temp:
                        temp.write(line)
                        temp.writelines(rows)
                    break

        if Path(os.path.join(os.getcwd(), "temp.json")).exists(): 
            with open(os.path.join(os.getcwd(), "temp.json"), 'r') as temp:
                data[file] = dict()
                data[file] = json.load(temp)
                os.remove(os.path.join(os.getcwd(), "temp.json"))

    return 1

parse_fio_data(f"../write-on-append-interference/data", data)

queue_depths = np.arange(1, 8)

write100 = [None] * len(queue_depths)
write100_iops = [None] * len(queue_depths)
write99 = [None] * len(queue_depths)
write99_iops = [None] * len(queue_depths)
write95 = [None] * len(queue_depths)
write95_iops = [None] * len(queue_depths)
write90 = [None] * len(queue_depths)
write90_iops = [None] * len(queue_depths)
write75 = [None] * len(queue_depths)
write75_iops = [None] * len(queue_depths)
write50 = [None] * len(queue_depths)
write50_iops = [None] * len(queue_depths)
write25 = [None] * len(queue_depths)
write25_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    if 'aflow_100' in key:
        x = int(value["jobs"][0]["job options"]["iodepth"]) - 1
        write100[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write100_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'aflow_99' in key:
        x = int(value["jobs"][1]["job options"]["iodepth"]) - 1
        write99[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write99_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_95' in key:
        x = int(value["jobs"][1]["job options"]["iodepth"]) - 1
        write95[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write95_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_90' in key:
        x = int(value["jobs"][1]["job options"]["iodepth"]) - 1
        write90[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write90_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_75' in key:
        x = int(value["jobs"][1]["job options"]["iodepth"]) - 1
        write75[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write75_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_50' in key:
        x = int(value["jobs"][1]["job options"]["iodepth"]) - 1
        write50[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write50_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000
    elif 'aflow_25' in key:
        x = int(value["jobs"][1]["job options"]["iodepth"]) - 1
        write25[x] = value["jobs"][1]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write25_iops[x] = value["jobs"][1]["finish"]["iops_mean"]/1000

print(f"Interference RMS 0% - 50% {get_interference_rms(write100_iops, write50_iops, write100, write50)}")

fig, ax = plt.subplots()

ax.plot(write100_iops, write100, markersize = 4, marker = '>', label="   0% write", color="#000000")
ax.plot(write99_iops, write99, markersize = 4, marker = 'x',   label="   1% write", color=CYAN)
ax.plot(write95_iops, write95, markersize = 4, marker = 'o',   label="   5% write", color=MAGENTA)
ax.plot(write90_iops, write90, markersize = 4, marker = '<',   label=" 10% write", color=OLIVE)
ax.plot(write75_iops, write75, markersize = 4, marker = '^',   label=" 25% write", color=ROSE)
ax.plot(write50_iops, write50, markersize = 4, marker = '*',   label=" 50% write", color=GREEN)
# ax.plot(write25_iops, write25, markersize = 4, marker = 'p',   label=" 75% finish")

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
# ax.legend(loc='best', handles=handles)
# ax.legend(loc='best')
ax.set_ylim(bottom=0, top=250)
ax.set_xlim(left=0, right=175)

ax.yaxis.set_ticks(np.arange(0, 300, 50), labels=['', '', '', '','',''])
#ax.get_yaxis().set_visible(False)
ax.xaxis.set_ticks(np.arange(0, 200, 25))

# ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')
plt.savefig('./plots/interference-micro-write-on-append.pdf', bbox_inches='tight')

In [ ]:
def parse_fio_data(data_path, data):
    if not os.path.exists(f'{data_path}') or \
            os.listdir(f'{data_path}') == []:
        print(f"No data in {data_path}")
        return 0

    for file in glob.glob(f'{data_path}/*'):
        if "bw" in file:
            continue
        with open(file, 'r') as f:
            for index, line in enumerate(f, 1):
                # Removing all fio logs in json file by finding first {
                if line.split() == []:
                    pass
                elif line.split()[0] == "{":
                    rows = f.readlines()
                    with open(os.path.join(os.getcwd(), "temp.json"), 'w+') as temp:
                        temp.write(line)
                        temp.writelines(rows)
                    break

        if Path(os.path.join(os.getcwd(), "temp.json")).exists(): 
            with open(os.path.join(os.getcwd(), "temp.json"), 'r') as temp:
                data[file] = dict()
                data[file] = json.load(temp)
                os.remove(os.path.join(os.getcwd(), "temp.json"))

    return 1

data = dict()
parse_fio_data(f"../append-on-write-interference/data", data)

queue_depths = np.arange(1, 8)

write100 = [None] * len(queue_depths)
write100_iops = [None] * len(queue_depths)
write99 = [None] * len(queue_depths)
write99_iops = [None] * len(queue_depths)
write95 = [None] * len(queue_depths)
write95_iops = [None] * len(queue_depths)
write90 = [None] * len(queue_depths)
write90_iops = [None] * len(queue_depths)
write75 = [None] * len(queue_depths)
write75_iops = [None] * len(queue_depths)
write50 = [None] * len(queue_depths)
write50_iops = [None] * len(queue_depths)
write25 = [None] * len(queue_depths)
write25_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    numjobs = int(re.search(r'\d+', key).group())
    if numjobs > 7:
        continue # We skip 14 as it turns out we were bottlenecked by not enough cpu cores, giving inaccurate results
    else:
        x = numjobs - 1
    if 'wflow_100' in key:
        write100[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write100_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_99' in key:
        write99[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write99_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_95' in key:
        write95[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write95_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_90' in key:
        write90[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write90_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_75' in key:
        write75[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write75_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_50' in key:
        write50[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write50_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_25' in key:
        write25[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write25_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000

print(f"Interference RMS 0% - 50% {get_interference_rms(write100_iops, write50_iops, write100, write50)}")

fig, ax = plt.subplots()

ax.plot(write100_iops, write100, markersize = 4, marker = '>', label=r"\textbf{0\%}", color="#000000")
ax.plot(write99_iops, write99, markersize = 4, marker = 'x',   label=r"\textbf{1\%}", color=CYAN)
ax.plot(write95_iops, write95, markersize = 4, marker = 'o',   label=r"\textbf{5\%}", color=MAGENTA)
ax.plot(write90_iops, write90, markersize = 4, marker = '<',   label=r"\textbf{10\%}", color=OLIVE)
ax.plot(write75_iops, write75, markersize = 4, marker = '^',   label=r"\textbf{25\%}", color=ROSE)
ax.plot(write50_iops, write50, markersize = 4, marker = '*',   label=r"\textbf{50\%}", color=GREEN)
# ax.plot(write25_iops, write25, markersize = 4, marker = 'p',   label=" 75% finish")

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
# ax.legend(loc='best', handles=handles)
ax.legend(loc='best', title=r'\textbf{Finish \%}')



ax.set_ylim(bottom=0, top=250)
ax.set_xlim(left=0, right=175)

ax.yaxis.set_ticks(np.arange(0, 300, 50))
ax.xaxis.set_ticks(np.arange(0, 200, 25))

ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')

plt.savefig('./plots/interference-micro-append-on-write.pdf', bbox_inches='tight')

In [ ]:
from pathlib import Path
def parse_fio_data(data_path, data):
    if not os.path.exists(f'{data_path}') or \
            os.listdir(f'{data_path}') == []:
        print(f"No data in {data_path}")
        return 0

    for file in glob.glob(f'{data_path}/*'):
        if "bw" in file:
            continue
        with open(file, 'r') as f:
            for index, line in enumerate(f, 1):
                # Removing all fio logs in json file by finding first {
                if line.split() == []:
                    pass
                elif line.split()[0] == "{":
                    rows = f.readlines()
                    with open(os.path.join(os.getcwd(), "temp.json"), 'w+') as temp:
                        temp.write(line)
                        temp.writelines(rows)
                    break

        if Path(os.path.join(os.getcwd(), "temp.json")).exists(): 
            with open(os.path.join(os.getcwd(), "temp.json"), 'r') as temp:
                data[file] = dict()
                data[file] = json.load(temp)
                os.remove(os.path.join(os.getcwd(), "temp.json"))

    return 1

data = dict()
parse_fio_data(f"../finish-on-write-interference/data", data)

queue_depths = np.arange(1, 8)

write100 = [None] * len(queue_depths)
write100_iops = [None] * len(queue_depths)
write99 = [None] * len(queue_depths)
write99_iops = [None] * len(queue_depths)
write95 = [None] * len(queue_depths)
write95_iops = [None] * len(queue_depths)
write90 = [None] * len(queue_depths)
write90_iops = [None] * len(queue_depths)
write75 = [None] * len(queue_depths)
write75_iops = [None] * len(queue_depths)
write50 = [None] * len(queue_depths)
write50_iops = [None] * len(queue_depths)
write25 = [None] * len(queue_depths)
write25_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    numjobs = int(re.search(r'\d+', key).group())
    if numjobs > 7:
        continue # We skip 14 as it turns out we were bottlenecked by not enough cpu cores, giving inaccurate results
    else:
        x = numjobs - 1
    if 'wflow_100' in key:
        write100[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write100_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_99' in key:
        write99[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write99_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_95' in key:
        write95[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write95_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_90' in key:
        write90[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write90_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_75' in key:
        write75[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write75_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_50' in key:
        write50[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write50_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_25' in key:
        write25[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write25_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000

print(f"Interference RMS 0% - 50% {get_interference_rms(write100_iops, write50_iops, write100, write50)}")

fig, ax = plt.subplots()

ax.plot(write100_iops, write100, markersize = 4, marker = '>', label=r'\textbf{0\%}', color="#000000",)
ax.plot(write99_iops, write99, markersize = 4, marker = 'x',   label=r'\textbf{1\%}', color=CYAN)
ax.plot(write95_iops, write95, markersize = 4, marker = 'o',   label=r'\textbf{5\%}', color=MAGENTA)
ax.plot(write90_iops, write90, markersize = 4, marker = '<',   label=r'\textbf{10\%}', color=OLIVE)
ax.plot(write75_iops, write75, markersize = 4, marker = '^',   label=r'\textbf{25\%}', color=ROSE)
ax.plot(write50_iops, write50, markersize = 4, marker = '*',   label=r'\textbf{50\%}', color=GREEN)
# ax.plot(write25_iops, write25, markersize = 4, marker = 'p',   label=" 75% finish")

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
# ax.legend(loc='best', handles=handles)
ax.legend(loc='best') # , title=r'\textbf{Finish \%}')
ax.set_ylim(bottom=0, top=150)
ax.set_xlim(left=0, right=175)

ax.yaxis.set_ticks(np.arange(0, 175, 25))
ax.xaxis.set_ticks(np.arange(0, 200, 25))

ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')

plt.savefig('./plots/interference-micro-finish-on-write.pdf', bbox_inches='tight')

In [ ]:
def parse_fio_data(data_path, data):
    if not os.path.exists(f'{data_path}') or \
            os.listdir(f'{data_path}') == []:
        print(f"No data in {data_path}")
        return 0

    for file in glob.glob(f'{data_path}/*'):
        if "bw" in file:
            continue
        with open(file, 'r') as f:
            for index, line in enumerate(f, 1):
                # Removing all fio logs in json file by finding first {
                if line.split() == []:
                    pass
                elif line.split()[0] == "{":
                    rows = f.readlines()
                    with open(os.path.join(os.getcwd(), "temp.json"), 'w+') as temp:
                        temp.write(line)
                        temp.writelines(rows)
                    break

        if Path(os.path.join(os.getcwd(), "temp.json")).exists(): 
            with open(os.path.join(os.getcwd(), "temp.json"), 'r') as temp:
                data[file] = dict()
                data[file] = json.load(temp)
                os.remove(os.path.join(os.getcwd(), "temp.json"))

    return 1

data = dict()
parse_fio_data(f"../finish-on-append-interference/data", data)

queue_depths = np.arange(1, 8)

write100 = [None] * len(queue_depths)
write100_iops = [None] * len(queue_depths)
write99 = [None] * len(queue_depths)
write99_iops = [None] * len(queue_depths)
write95 = [None] * len(queue_depths)
write95_iops = [None] * len(queue_depths)
write90 = [None] * len(queue_depths)
write90_iops = [None] * len(queue_depths)
write75 = [None] * len(queue_depths)
write75_iops = [None] * len(queue_depths)
write50 = [None] * len(queue_depths)
write50_iops = [None] * len(queue_depths)
write25 = [None] * len(queue_depths)
write25_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    numjobs = int(re.search(r'\d+', key).group())
    if numjobs > 7:
        continue # We skip 14 as it turns out we were bottlenecked by not enough cpu cores, giving inaccurate results
    else:
        x = numjobs - 1
    if 'wflow_100' in key:
        write100[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write100_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_99' in key:
        write99[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write99_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_95' in key:
        write95[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write95_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_90' in key:
        write90[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write90_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_75' in key:
        write75[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write75_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_50' in key:
        write50[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write50_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000
    elif 'wflow_25' in key:
        write25[x] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
        write25_iops[x] = value["jobs"][0]["finish"]["iops_mean"]/1000

print(f"Interference RMS 0% - 50% {get_interference_rms(write100_iops, write50_iops, write100, write50)}")

fig, ax = plt.subplots()

ax.plot(write100_iops, write100, markersize = 4, marker = '>', label=r'\textbf{0\%}', color="#000000",)
ax.plot(write99_iops, write99, markersize = 4, marker = 'x',   label=r'\textbf{1\%}', color=CYAN)
ax.plot(write95_iops, write95, markersize = 4, marker = 'o',   label=r'\textbf{5\%}', color=MAGENTA)
ax.plot(write90_iops, write90, markersize = 4, marker = '<',   label=r'\textbf{10\%}', color=OLIVE)
ax.plot(write75_iops, write75, markersize = 4, marker = '^',   label=r'\textbf{25\%}', color=ROSE)
ax.plot(write50_iops, write50, markersize = 4, marker = '*',   label=r'\textbf{50\%}', color=GREEN)
# ax.plot(write25_iops, write25, markersize = 4, marker = 'p',   label=" 75% finish")

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
# ax.legend(loc='best', handles=handles)
# ax.legend(loc='best', title=r'\textbf{Finish \%}')
ax.set_ylim(bottom=0, top=150)
ax.set_xlim(left=0, right=175)

ax.yaxis.set_ticks(np.arange(0, 175, 25))
ax.xaxis.set_ticks(np.arange(0, 200, 25))

ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')

plt.savefig('./plots/interference-micro-finish-on-append.pdf', bbox_inches='tight')

In [ ]:
def parse_fio_data(data_path, data):
    if not os.path.exists(f'{data_path}') or \
            os.listdir(f'{data_path}') == []:
        print(f"No data in {data_path}")
        return 0

    for file in glob.glob(f'{data_path}/*'):
        if "bw" in file:
            continue
        with open(file, 'r') as f:
            for index, line in enumerate(f, 1):
                # Removing all fio logs in json file by finding first {
                if line.split() == []:
                    pass
                elif line.split()[0] == "{":
                    rows = f.readlines()
                    with open(os.path.join(os.getcwd(), "temp.json"), 'w+') as temp:
                        temp.write(line)
                        temp.writelines(rows)
                    break

        if Path(os.path.join(os.getcwd(), "temp.json")).exists(): 
            with open(os.path.join(os.getcwd(), "temp.json"), 'r') as temp:
                data[file] = dict()
                data[file] = json.load(temp)
                os.remove(os.path.join(os.getcwd(), "temp.json"))
    return 1

data = dict()
parse_fio_data(f"../finish-on-read-inteference/data", data)

queue_depths = np.arange(1, 9)

read100 = [None] * len(queue_depths)
read100_iops = [None] * len(queue_depths)
read99 = [None] * len(queue_depths)
read99_iops = [None] * len(queue_depths)
read95 = [None] * len(queue_depths)
read95_iops = [None] * len(queue_depths)
read90 = [None] * len(queue_depths)
read90_iops = [None] * len(queue_depths)
read75 = [None] * len(queue_depths)
read75_iops = [None] * len(queue_depths)
read50 = [None] * len(queue_depths)
read50_iops = [None] * len(queue_depths)
read25 = [None] * len(queue_depths)
read25_iops = [None] * len(queue_depths)

for key, value in data.items():
    if "bw" in key:
        continue
    x = int(math.log2(int(value["jobs"][1]["job options"]["iodepth"])))
    if 'rflow_100' in key:
        read100[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read100_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_99' in key:
        read99[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read99_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_95' in key:
        read95[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read95_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_90' in key:
        read90[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read90_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_75' in key:
        read75[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read75_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_50' in key:
        read50[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read50_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000
    elif 'rflow_25' in key:
        read25[x] = value["jobs"][1]["read"]["lat_ns"]["percentile"]["95.000000"]/1000
        read25_iops[x] = value["jobs"][1]["read"]["iops_mean"]/1000


print(f"Interference RMS 0% - 50% {get_interference_rms(read100_iops, read50_iops, read100, read50)}")


fig, ax = plt.subplots()

ax.plot(read100_iops, read100, markersize = 4, marker = '>', label=r'\textbf{0\%}', color="#000000",)
ax.plot(read99_iops, read99, markersize = 4, marker = 'x',   label=r'\textbf{1\%}', color=CYAN)
ax.plot(read95_iops, read95, markersize = 4, marker = 'o',   label=r'\textbf{5\%}', color=MAGENTA)
ax.plot(read90_iops, read90, markersize = 4, marker = '<',   label=r'\textbf{10\%}', color=OLIVE)
ax.plot(read75_iops, read75, markersize = 4, marker = '^',   label=r'\textbf{25\%}', color=ROSE)
ax.plot(read50_iops, read50, markersize = 4, marker = '*',   label=r'\textbf{50\%}', color=GREEN)
# ax.plot(write25_iops, write25, markersize = 4, marker = 'p',   label=" 75% finish")

fig.tight_layout()
ax.grid(which='major', linestyle='dashed', linewidth='1')
ax.set_axisbelow(True)
# ax.legend(loc='best', handles=handles)
# ax.legend(loc='best', title=r'\textbf{Finish \%}')
ax.set_ylim(bottom=0, top=150)
ax.set_xlim(left=0, right=175)

ax.yaxis.set_ticks(np.arange(0, 2400, 250))
ax.xaxis.set_ticks(np.arange(0, 350, 50))

ax.set_ylabel(r'\textbf{P95 latency ($\mu$s)}')
ax.set_xlabel(r'\textbf{Throughput (KIOPS)}')

ax.annotate(r'\textbf{B}',
            xy=(read50_iops[-1],read50[-1]-90), xycoords='data',
            xytext=(55, -0), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='center', verticalalignment='bottom')

ax.annotate(r'\textbf{A}',
            xy=(read100_iops[-1],read100[-1]+65), xycoords='data',
            xytext=(0, 20), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='center', verticalalignment='bottom')


print(read50_iops[-1],read50[-1], read100_iops[-1],read100[-1] )




plt.savefig('./plots/interference-micro-finish-on-read.pdf', bbox_inches='tight')

In [ ]:
# TODO: retrieve these values from ZenFS tracing for final calculation
WRITE_INTERFERENCE_GAMMA = 0.5
RESET_INTERFERENCE_DELTA = 0.5

RESET_ON_WRITE_RMS = 0.241 # From our benchmark reset-on-write-interference, the retrieved RMS value
WRITE_ON_RESET_RMS = 0.072 # From our benchmark write-on-reset-interference, the retrieved RMS value

# Switch to Type 1 Fonts.
plt.rcParams['text.usetex'] = True
plt.rc('font', **{'family': 'serif', 'serif': ['Times']})

matplotlib_color = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5']
m_color_index = 0

matplotlib_colors = [
    'blue', 'green', 'red', 'cyan', 'magenta', 'yellw', 'white'
]

dot_style = [
    '+',
    'X',
    'o',
    'v',
    's',
    'P',
]

# Global parameters
linewidth = 4
markersize = 15

datalabel_size = 36
datalabel_va = 'bottom'
axis_tick_font_size = 46
axis_label_font_size = 52
legend_font_size = 46

heatmap_axis_tick_font_size = 26
heatmap_data_tag_size = 16

def parse_fio_data(data_path, data):
    if not os.path.exists(f'{data_path}') or \
            os.listdir(f'{data_path}') == []:
        print(f"No data in {data_path}")
        return 0
    
    for file in glob.glob(f'{data_path}/*'):
        if "bw" in file:
            continue
        with open(file, 'r') as f:
            for index, line in enumerate(f, 1):
                # Removing all fio logs in json file by finding first {
                if line.split()[0] == "{":
                    rows = f.readlines()
                    with open(os.path.join(os.getcwd(), "temp.json"), 'w+') as temp:
                        temp.write(line)
                        temp.writelines(rows)
                    break

        if Path(os.path.join(os.getcwd(), "temp.json")).exists(): 
            with open(os.path.join(os.getcwd(), "temp.json"), 'r') as temp:
                data[file] = dict()
                data[file] = json.load(temp)
                os.remove(os.path.join(os.getcwd(), "temp.json"))

    return 1

def init_baseline(baseline):
    val = baseline[0]

    i = 0
    for value in baseline:
        baseline[i] = val
        i += 1

def parse_write_baseline(write_baseline_iops, write_baseline_lat):
     for conf_key, conf_value in data.items():

        for key, value in conf_value.items():
            if conf_key == "baseline":
                if "reset_baseline" in key:
                    continue
                elif "bw" in key:
                    continue
                else:
                    head, tail = os.path.split(key)
                    numjobs = int(re.search(r'\d+', tail).group())
                    write_baseline_iops[numjobs - 1] =  value["jobs"][0]["finish"]["iops_mean"]/1000
                    write_baseline_lat[numjobs - 1] = value["jobs"][0]["finish"]["lat_ns"]["percentile"]["95.000000"]/1000
            else:
                continue

def parse_reset_baseline(reset_baseline_iops, reset_baseline_write):
     for conf_key, conf_value in data.items():

        for key, value in conf_value.items():
            if conf_key == "baseline":
                if "reset_baseline" in key:
                    reset_baseline_iops[0] = value["jobs"][1]["ZNS Reset"]["iops_mean"]
                    reset_baseline_lat[0] = value["jobs"][1]["ZNS Reset"]["lat_ns"]["percentile"]["95.000000"]/1000 # TODO: change this to clat after rerunning
                    init_baseline(reset_baseline_iops)
                    init_baseline(reset_baseline_lat)

                    return
                else:
                    continue
            else:
                continue


def get_matrix_col(val):
    """Get the config index for the col which represents the reset latency"""
    match(val):
        case 16:
            return 3
        case 32:
            return 2
        case 64:
            return 1
        case 128:
            return 0

def get_matrix_row(val):
    """Get the config index for the row which represents the write ratio"""
    match(val):
        case 200:
            return 0
        case 2000:
            return 1
        case 20000:
            return 2
        case 200000:
            return 3
        
def generate_heatmap(config_interference, job, max):
    reset_latency = [16, 32, 64, 128]
    write_ratio    = [200, 2000, 20000, 200000]

    cmap = sns.color_palette('rocket_r', as_cmap=True).copy()
    cmap.set_under('#88CCEE')
    
    ax = sns.heatmap(config_interference, 
                    linewidth=0.1, 
                    xticklabels=True, 
                    cmap=cmap, 
                    yticklabels=True, 
                    clip_on=False, 
                    # cbar_kws={'shrink': 0.8, 'extend': 'min', 'extendrect': True, 'format': '%d Blocks (512B)'}, 
                    square=True, 
                    cbar=True, 
                    vmin=0,
                    vmax=max,
                    linecolor='black',  annot_kws={"size": 20})

    ax.set_xticks([x+0.5 for x in np.arange(len(write_ratio))])
    ax.set_xticklabels(labels=write_ratio,
                fontsize=25)
    ax.xaxis.set_label_position('top') 
    ax.xaxis.tick_top()

    ax.set_yticks([x+0.5 for x in np.arange(len(reset_latency))])
    ax.set_yticklabels(labels=reset_latency[::-1],
                    fontsize=25)

    plt.setp(ax.get_xticklabels(),
                rotation=45,
                ha="left",
                rotation_mode="anchor")
    
    plt.setp(ax.get_yticklabels(),
                rotation=45,
                ha="right",
                rotation_mode="anchor")
    
    ax.set_ylabel(r"\textbf{Epoch interval (ms)}", fontsize=25)
    ax.set_xlabel(r"\textbf{Request tokens}", fontsize=25)
    
    for i in range(len(config_interference)):
        for j in range(len(config_interference[0])):
            text = round(config_interference[i][j], 3)
            text = r'\textbf{' + str(text) + '}'
            if config_interference[i][j] >= 0.9:
                color = 'w'
            elif config_interference[i][j] == 0:
                color = 'black'
            else:
                color = 'black'
            text = ax.text(j+0.5,
                            i+0.5,
                            text,
                            ha="center",
                            va="center",
                            color=color,
                            fontsize=heatmap_data_tag_size)

    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=20)

    plt.savefig(f"{file_path}/figures/configuration-interference-{job}.pdf", bbox_inches="tight")
    plt.savefig(f"{file_path}/figures/configuration-interference-{job}.png", bbox_inches="tight")
    fig, ax = plt.subplots()

    #     plt.clf()
#     plt.close()

file_path = '../zinc-configuration'

data = dict()
for dir in glob.glob(f'{file_path}/*'):
    dir = dir.split('/')[-1]
    if dir == "data":
        data["baseline"] = dict()
        parse_fio_data(f"{file_path}/{dir}", data["baseline"])
    elif "data" in dir:
        config = re.findall(r'\d+', dir)
        config_string = f"reset_lat_{config[0]}-write_ratio_{config[1]}"
        data[config_string] = dict()
        data[config_string]["reset_limit_val"] = config[0]
        data[config_string]["write_ratio_val"] = config[1]
        parse_fio_data(f"{file_path}/{dir}", data[config_string])

os.makedirs(f"{file_path}/figures", exist_ok=True)

queue_depths = np.arange(1, 8)
lowest_interference = (None, None)

reset_baseline_iops = [None] * len(queue_depths)
reset_baseline_lat = [None] * len(queue_depths)
write_baseline_iops = [None] * len(queue_depths)
write_baseline_lat = [None] * len(queue_depths)
config_reset_limit = []
config_write_ratio = []
config_interference = np.zeros(shape=(4, 4))
config_interference_write = np.zeros(shape=(4, 4))
config_interference_reset = np.zeros(shape=(4, 4))

parse_write_baseline(write_baseline_iops, write_baseline_lat)
parse_reset_baseline(reset_baseline_iops, reset_baseline_lat)

for conf_key, conf_value in data.items():
    write_iops = [None] * len(queue_depths)
    write_lat = [None] * len(queue_depths)
    reset_iops = [None] * len(queue_depths)
    reset_lat = [None] * len(queue_depths)

    for key, value in conf_value.items():
        if conf_key == "baseline":
           continue
        elif "reset_limit_val" in key or "write_ratio_val" in key:
            continue
        else:
            head, tail = os.path.split(key)
            numjobs = int(re.search(r'\d+', tail).group())
            x = numjobs - 1
            write_iops[x] = value["jobs"][2]["finish"]["iops_mean"]/1000
            write_lat[x] = value["jobs"][2]["finish"]["clat_ns"]["percentile"]["95.000000"]/1000
            reset_iops[x] = value["jobs"][1]["ZNS Reset"]["iops_mean"]
            reset_lat[x] = value["jobs"][1]["ZNS Reset"]["clat_ns"]["percentile"]["95.000000"]/1000

    # While debugging skip all that are ongoing and don't have all data
    if not None in write_iops:
        write_interference = get_interference_rms(write_baseline_iops, write_iops, write_baseline_lat, write_lat)
        reset_interference = get_interference_rms(reset_baseline_iops, reset_iops, reset_baseline_lat, reset_lat)

        print("-------------------------------------------------------------------------------------")
        print(f"Config {conf_key: >40} WRITE Interference RMS {write_interference: >20.15f}")
        print(f"Config {conf_key: >40} RESET Interference RMS {reset_interference: >20.15f}")

        interference = WRITE_INTERFERENCE_GAMMA * write_interference + RESET_INTERFERENCE_DELTA * reset_interference
        print(f"Config {conf_key : >40} Interference RMS {interference : >26.15f}")
        print("-------------------------------------------------------------------------------------")

        # config_reset_limit.append(int(conf_value["reset_limit_val"]))
        # config_write_ratio.append(int(conf_value["write_ratio_val"]))
        config_interference[get_matrix_col(int(conf_value["reset_limit_val"]))][get_matrix_row(int(conf_value["write_ratio_val"]))] = interference

        config_interference_write[get_matrix_col(int(conf_value["reset_limit_val"]))][get_matrix_row(int(conf_value["write_ratio_val"]))] = write_interference
        config_interference_reset[get_matrix_col(int(conf_value["reset_limit_val"]))][get_matrix_row(int(conf_value["write_ratio_val"]))] = reset_interference

        # This one is to calculate relative gains respective to the benchmarked reset-on-write-interference RMS
        # rms_repesctive_change = float(RESET_ON_WRITE_RMS) / write_interference
        # config_interference_write[get_matrix_col(int(conf_value["reset_limit_val"]))][get_matrix_row(int(conf_value["write_ratio_val"]))] = rms_repesctive_change

        # This one is to calculate relative gains respective to the benchmarked write-on-reset-interference RMS
        # rms_repesctive_change = float(WRITE_ON_RESET_RMS) / reset_interference
        # config_interference_reset[get_matrix_col(int(conf_value["reset_limit_val"]))][get_matrix_row(int(conf_value["write_ratio_val"]))] = rms_repesctive_change


        inter = lowest_interference[1]
        if inter == None:
            lowest_interference = (conf_key, interference)
        elif interference < inter:
                lowest_interference = (conf_key, interference)

        fig, ax = plt.subplots()

        ax.plot(write_baseline_iops, write_baseline_lat, markersize = 4, marker = '>', label="   0% reset")
        ax.plot(write_iops, write_lat, markersize = 4, marker = '*', label=" 50% reset")

        fig.tight_layout()
        ax.grid(which='major', linestyle='dashed', linewidth='1')
        ax.set_axisbelow(True)
        # ax.legend(loc='best', handles=handles)
        ax.legend(loc='best')
        ax.set_ylim(bottom=0, top=140)
        ax.set_xlim(left=0)
        ax.set_ylabel("p95 write Latency (usec)")
        ax.set_xlabel("Total IOPS (x1000)")
        plt.savefig(f"{file_path}/figures/loaded_write_latency-{conf_key}.pdf", bbox_inches="tight")
        plt.savefig(f"{file_path}/figures/loaded_write_latency-{conf_key}.png", bbox_inches="tight")
        plt.clf()
        plt.close()

print("\n=====================================================================================")
print(f"{Fore.GREEN}Lowest{Style.RESET_ALL} {lowest_interference[0] : >40} Interference RMS {lowest_interference[1]:>26.15f}")


generate_heatmap(config_interference_write, "write", 2)
generate_heatmap(config_interference_reset, "reset", 25)
generate_heatmap(config_interference, "combined", 15)

In [ ]:
prio_run_dat=[]
for prio in range(0,6):
    prio_run_dat_tmp = []
    for run in range(1,3):
        load1 = np.genfromtxt(f"../zinc-cdf/data/{run}_{prio}_reset_lat.6.log", delimiter=",", usemask=True)
        prio_run_dat_tmp = prio_run_dat_tmp + [load[1]/1000000 for load in load1]   
    prio_run_dat.append(prio_run_dat_tmp)

fig, ax = plt.subplots()

colors = ["#000000", CYAN, MAGENTA, OLIVE, ROSE, GREEN]
ite=0
for prio in prio_run_dat:
    data = prio
    count, bins_count = np.histogram(data, bins=len(prio)) 
    pdf = count / sum(count) 
    cdf = np.cumsum(pdf) 
    plt.plot(bins_count[1:], cdf, label=r"\textbf{" + str(ite) + "}",color=colors[ite])
    ite = ite + 1


plt.legend(title=r"\textbf{Max epoch holds}", loc='lower right',ncol=2)

ax.set_ylim(bottom=0, top=1)
ax.set_xlim(left=0, right=384)
ax.yaxis.set_ticks([0,0.25,0.5,0.75,1])
ax.xaxis.set_ticks(np.arange(0, 1200, 100))

plt.setp(ax.get_xticklabels(),
            rotation=45,
            ha="right",
            rotation_mode="anchor")

ax.set_ylabel(r'\textbf{Cumulative probality}')
ax.set_xlabel(r'\textbf{Rese latency (ms)}')

plt.axhline(y = 0.95, color = 'r', linestyle = '--') 
ax.annotate(r"\textbf{$\sim$64ms}", xy=(335, 0.70),  xytext=(420, 0.68), arrowprops=dict(arrowstyle="<-"))
plt.savefig(f"./plots/cdf-reset-epoch-holds.pdf", bbox_inches="tight")